<a href="https://colab.research.google.com/github/saivinaya279/netflix/blob/main/netflixipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# this is just to know how much time will it take to run this entire ipython notebook
from datetime import datetime
# globalstart = datetime.now()
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('nbagg')

import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})

import seaborn as sns
sns.set_style('whitegrid')
import os
from scipy import sparse
from scipy.sparse import csr_matrix

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import random

 # Exploratory Data Analysis

3.1 Preprocessing
3.1.1 Converting / Merging whole data to required format: u_i, m_j, r_ij

In [5]:
import os
print(os.getcwd())


/content


In [6]:
import os
from datetime import datetime

# Delete 'data.csv' if it already exists
if os.path.exists('data.csv'):
    start = datetime.now()

# Open a new file for writing the combined data
data = open('data.csv', mode='w')

# Files to process (all inside data_folder)
files = [
    'combined_data_1.txt',
    'combined_data_2.txt',
    'combined_data_3.txt',
    'combined_data_4.txt'
]

# Go through each file and process it
for file in files:
    if os.path.exists(file):
        print(f"Reading ratings from {file}...")
        with open(file) as f:
            for line in f:
                line = line.strip()
                if line.endswith(':'):
                    movie_id = line.replace(':', '')
                else:
                    row = [x for x in line.split(',')]
                    row.insert(0, movie_id)  # Insert movie ID at the beginning
                    data.write(','.join(row) + '\n')
        print("Done.\n")
    else:
        print(f"⚠️ File not found: {file}")

data.close()

print("✅ All available data processed.")
print("⏱️ Time taken:", datetime.now() - start)


Reading ratings from combined_data_1.txt...
Done.

Reading ratings from combined_data_2.txt...
Done.

Reading ratings from combined_data_3.txt...
Done.

Reading ratings from combined_data_4.txt...
Done.

✅ All available data processed.
⏱️ Time taken: 0:01:14.564308


In [7]:
import pandas as pd

print("Creating the dataframe from data.csv file..")
df = pd.read_csv('data.csv', sep=',',
                 names=['movie', 'user', 'rating', 'date'])

# Convert date column, infer format, coerce invalid parsing to NaT
df['date'] = pd.to_datetime(df['date'], errors='coerce', infer_datetime_format=True)

print("Done.\n")

# Optional: remove rows with invalid dates if needed
df = df.dropna(subset=['date'])

print("Sorting the dataframe by date..")
df.sort_values(by='date', inplace=True)
print("Done..")


Creating the dataframe from data.csv file..


/tmp/ipython-input-2842833079.py:8: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date'] = pd.to_datetime(df['date'], errors='coerce', infer_datetime_format=True)


Done.

Sorting the dataframe by date..
Done..


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57777598 entries, 36022036 to 15419576
Data columns (total 4 columns):
 #   Column  Dtype         
---  ------  -----         
 0   movie   int64         
 1   user    int64         
 2   rating  float64       
 3   date    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 2.2 GB


In [9]:
df.describe()

,movie,user,rating,date
count,5.777760e+07,5.777760e+07,5.777760e+07,57777598
mean,7.818241e+03,1.322404e+06,3.608001e+00,2004-10-09 05:53:03.561750272
min,1.000000e+00,6.000000e+00,1.000000e+00,1999-11-11 00:00:00
25%,2.782000e+03,6.610980e+05,3.000000e+00,2004-05-03 00:00:00
50%,6.829000e+03,1.318925e+06,4.000000e+00,2005-01-22 00:00:00
75%,1.163800e+04,1.984352e+06,4.000000e+00,2005-07-09 00:00:00
max,1.542500e+04,2.649429e+06,5.000000e+00,2005-12-31 00:00:00
std,4.882936e+03,7.645334e+05,1.086041e+00,NaN


In [10]:
df.describe() ['rating']

,rating
count,5.777760e+07
mean,3.608001e+00
min,1.000000e+00
25%,3.000000e+00
50%,4.000000e+00
75%,4.000000e+00
max,5.000000e+00
std,1.086041e+00


In [11]:
df.head()

,movie,user,rating,date
36022036,10341,510180,4.0,1999-11-11
9056171,1798,510180,5.0,1999-11-11
55701390,15105,510180,4.0,1999-11-11
52841753,14660,510180,2.0,1999-11-11
38288821,10774,510180,3.0,1999-11-11


In [12]:
print("No of nan values in our dataframe:",sum(df.isnull().any()))

No of nan values in our dataframe: 0


In [13]:
dup_bool = df.duplicated(['movie','user','rating'])
dups=sum(dup_bool)
print("There are {} duplicate rating entries in the data...".format(dups))

There are 0 duplicate rating entries in the data...


# Basic statsistics (#ratings,#usersand #movies)


In [14]:
import numpy as np

print("Total data ")
print("-" * 50)
print("\nTotal no of ratings :", df.shape[0])
print("Total No of Users:", len(np.unique(df['user'])))
print("Total No of Movies:",len(np.unique(df.movie)))


Total data 
--------------------------------------------------

Total no of ratings : 57777598
Total No of Users: 478794
Total No of Movies: 10109


# spliting data into Train and Test(80:20) (80-train data,20-test data)

In [15]:
import os
import pandas as pd

# Create train.csv if not exists
if not os.path.isfile("train.csv"):
    df.iloc[:int(df.shape[0] * 0.80)].to_csv("train.csv", index=False)

# Create test.csv if not exists
if not os.path.isfile("test.csv"):
    df.iloc[int(df.shape[0] * 0.80):].to_csv("test.csv", index=False)

# Read back the files
train_df = pd.read_csv("train.csv", parse_dates=['date'])
test_df = pd.read_csv("test.csv")


# basic statistics in Train data(#ratings,#users,and #Movies)


In [16]:
import numpy as np

print("Training data ")
print("-" * 50)
print("\nTotal no of ratings :", train_df.shape[0])
print("Total No of Users:", len(np.unique(train_df.user)))
print("Total No of Movies:",len(np.unique(train_df.movie)))



Training data 
--------------------------------------------------

Total no of ratings : 46222078
Total No of Users: 402272
Total No of Movies: 9907


# basic statistics in Test data(#ratings,#users,and #Movies)


In [17]:
print("Test data ")
print("-" * 50)
print("\nTotal no of ratings :", test_df.shape[0])
print("Total No of Users:", len(np.unique(test_df.user)))
print("Total No of Movies:",len(np.unique(test_df.movie)))



Test data 
--------------------------------------------------

Total no of ratings : 11555520
Total No of Users: 331136
Total No of Movies: 10102


# Exploratory Data Analysis on Train data

In [18]:
# method to make y-axis more redable
def human(num,units='M'):
    units=units.lower()
    num=float(num)
    if units=='k':
        return str(num/10**3)+"K"
    elif units=='m':
        return str(num/10**6) +"M"
    elif units =='b':
        return str(num/10**9) + "B"

# Distribution of ratings

In [20]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Aggregate ratings first (count occurrences)
rating_counts = train_df['rating'].value_counts().sort_index()

# 2. Plot aggregated data
fig, ax = plt.subplots()
plt.title('Distribution of Ratings over Training Dataset', fontsize=15)

sns.barplot(x=rating_counts.index, y=rating_counts.values, ax=ax)

# 3. Format y-axis in millions
ax.set_yticklabels([f'{int(item/1_000_000)}M' for item in ax.get_yticks()])

ax.set_ylabel('No. of Ratings (Millions)')
ax.set_xlabel('Rating')
plt.show()


<IPython.core.display.Javascript object>

/tmp/ipython-input-927626692.py:14: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels([f'{int(item/1_000_000)}M' for item in ax.get_yticks()])


In [22]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Aggregate ratings first (count occurrences)
rating_counts = train_df['rating'].value_counts().sort_index()

# 2. Plot aggregated data
fig, ax = plt.subplots()
plt.title('Distribution of Ratings over Training Dataset', fontsize=15)

sns.barplot(x=rating_counts.index, y=rating_counts.values, ax=ax)

# 3. Set fixed y-ticks and format in millions
yticks = ax.get_yticks()
ax.set_yticks(yticks)  # explicitly set fixed ticks
ax.set_yticklabels([f'{int(item/1_000_000)}M' for item in yticks])

ax.set_ylabel('No. of Ratings (Millions)')
ax.set_xlabel('Rating')
plt.show()


<IPython.core.display.Javascript object>